In [1]:
import pandas as pd
import numpy as np
import png
import time

In [4]:
df=pd.read_csv("../all_ratings.csv")
df.head()

,movie_id,user_id,rating
0,4500,2532865,4
1,4500,573364,3
2,4500,1696725,3
3,4500,1253431,3
4,4500,1265574,2


In [34]:
titles_df = pd.read_csv("../flask/static/movie_titles.csv",header = None, names = ['movie_id', 'year', 'title'], usecols=[0,1,2],encoding = "ISO-8859-1")
titles_df.head()

,movie_id,year,title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [4]:
mvc=df["movie_id"].value_counts()

In [5]:
top_1k_ids =list(mvc.head(1000).keys())

In [6]:
def get_title(df, movie_id):
    return df[df.movie_id==movie_id].iloc[0].title

In [7]:
[get_title(titles_df, x) for x in top_1k_ids]

['Miss Congeniality',
 'Independence Day',
 'The Patriot',
 'The Day After Tomorrow',
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 'Pretty Woman',
 'Forrest Gump',
 'The Green Mile',
 'Con Air',
 'Twister',
 'Sweet Home Alabama',
 'Pearl Harbor',
 'Armageddon',
 'The Rock',
 'What Women Want',
 'Bruce Almighty',
 "Ocean's Eleven",
 'The Bourne Identity',
 'The Italian Job',
 'I',
 'American Beauty',
 'How to Lose a Guy in 10 Days',
 'Lethal Weapon 4',
 'Shrek 2',
 'Lost in Translation',
 'Top Gun',
 'Pulp Fiction',
 'Gone in 60 Seconds',
 'The Sixth Sense',
 'Lord of the Rings: The Two Towers',
 'Men of Honor',
 'Gladiator',
 'Lord of the Rings: The Fellowship of the Ring',
 'Sister Act',
 'Double Jeopardy',
 'Two Weeks Notice',
 'The Royal Tenenbaums',
 'Troy',
 'National Treasure',
 '50 First Dates',
 'Indiana Jones and the Last Crusade',
 'My Big Fat Greek Wedding',
 'Mystic River',
 'Titanic',
 'Dirty Dancing',
 'Catch Me If You Can',
 'Finding Nemo (Widescreen)',
 '

In [8]:
titles_df.query(f"movie_id in {top_1k_ids}")

,movie_id,year,title
27,28,2002.0,Lilo and Stitch
29,30,2003.0,Something's Gotta Give
110,111,2003.0,Duplex (Widescreen)
142,143,1997.0,The Game
174,175,1992.0,Reservoir Dogs
190,191,2003.0,X2: X-Men United
196,197,2004.0,Taking Lives
198,199,1978.0,The Deer Hunter
240,241,1959.0,North by Northwest
256,257,1973.0,Charlotte's Web


In [12]:
titles_df.query(f"movie_id in {top_1k_ids}")[["movie_id","title"]].to_csv("top_1k_titles.csv",index=False)

In [2]:
titles_1k = pd.read_csv("top_1k_titles.csv")

In [3]:
ids_1k = list(titles_1k["movie_id"])

In [5]:
movies = {}
for i in ids_1k:
    movies[i] = df[df["movie_id"]==i].drop(["movie_id"],axis=1).set_index("user_id")

In [39]:
def get_title(df, movie_id):
    return df[df.movie_id==movie_id].iloc[0].title

In [6]:
def rough_means_v1(df, id1, id2, movie_dict=None, col_norm=50, low_clip=0, sym=False):
    if id1==id2:
        if sym:
            return [0,0,0],[0,0,0]
        else:
            return [0,0,0]
        
    if movie_dict is None:
        df1 = df[df["movie_id"]==id1].drop(["movie_id"],axis=1)
        df2 = df[df["movie_id"]==id2].drop(["movie_id"],axis=1)
    else:
        df1 = movie_dict[id1]
        df2 = movie_dict[id2]
    
    df_join = df1.join(df2, how="inner", lsuffix="_x", rsuffix="_y")
    
    if sym:
        return rough_means_df(df_join, True, col_norm, low_clip), rough_means_df(df_join, False, col_norm, low_clip)
    else:
        return rough_means_df(df_join, True, col_norm, low_clip)

In [7]:
def rough_means_df(df_join, col_x=True, col_norm=50, low_clip=0):
    total = len(df_join)
    if total == 0 or total < low_clip:
        return [0,0,0]
    
    rx = df_join["rating_x"] if col_x else df_join["rating_y"]
    ry = df_join["rating_y"] if col_x else df_join["rating_x"]
    
    low = df_join[rx<3]
    med = df_join[rx==3]
    high = df_join[rx>3]
    
    result = []
    
    for x in [(low,2),(med,1),(high,2)]:
        if len(x[0])==0:
            result.append(0)
        else:
            if col_x:
                result.append(np.interp(x[0]["rating_y"].mean(),[1,5],[0,1])*np.interp(len(x[0]),[0,col_norm*x[1]],[0,1]))
            else:
                result.append(np.interp(x[0]["rating_x"].mean(),[1,5],[0,1])*np.interp(len(x[0]),[0,col_norm*x[1]],[0,1]))

    return result

In [13]:
def rough_means_matrix_sparse(id_list, movie_dict, col_norm=50, low_clip=0, progress=False, prog_interval=100):
    _range=len(id_list)
    
    matrix = [[[0,0,0]]*_range for x in range(_range)]

    counter=0
    t0=time.time()
    for i in range(_range-1):
        for j in range(i+1, _range):
            if progress:
                if counter%prog_interval==0:
                    t1=time.time()
                    print(f"{counter} records complete... ({round(t1-t0,3)} s elapsed)")
                counter+=1
                
            #Compute rough_means for (i,j) and (j,i)
            x1, x2 = rough_means_v1(None, id_list[i], id_list[j], movie_dict, col_norm, low_clip, True)
            matrix[i][j] = x1
            matrix[j][i] = x2
            
    return matrix

In [15]:
def matrix_to_png(matrix, range_=(0,1), arraylike=True):
    num_rows = len(matrix)
    num_cols = max(len(matrix[0]), len(matrix[-1]))
    if arraylike:
        num_channels = max(len(matrix[0][0]),3)
    else:
        num_channels = 1

    img_array = np.reshape(np.array([0,0,0]*num_rows*num_cols),[num_rows, num_cols*3])
    
    def interp(x):
        return np.interp(x, range_,[0,255])
    
    for i in range(num_rows):
        num_cols_i = len(matrix[i])
        for j in range(num_cols_i):
            if not arraylike:
                for k in range(3):
                    img_array[i][3*j+k]=interp(matrix[i][j])
            else:
                for k in range(num_channels):
                    img_array[i][3*j+k]=interp(matrix[i][j][k])
                    
    img_array = np.vstack(map(np.uint8, img_array))
    
    return png.from_array(img_array,'RGB')

In [14]:
means_matrix_100=rough_means_matrix_sparse(ids_1k[:100],movies,progress=True)

0 records complete... (0.0 s elapsed)
100 records complete... (1.83 s elapsed)
200 records complete... (4.31 s elapsed)
300 records complete... (5.79 s elapsed)
400 records complete... (7.64 s elapsed)
500 records complete... (10.1 s elapsed)
600 records complete... (12.33 s elapsed)
700 records complete... (14.31 s elapsed)
800 records complete... (16.03 s elapsed)
900 records complete... (17.57 s elapsed)
1000 records complete... (19.1 s elapsed)
1100 records complete... (20.72 s elapsed)
1200 records complete... (22.39 s elapsed)
1300 records complete... (23.97 s elapsed)
1400 records complete... (26.08 s elapsed)
1500 records complete... (28.06 s elapsed)
1600 records complete... (30.11 s elapsed)
1700 records complete... (31.83 s elapsed)
1800 records complete... (33.51 s elapsed)
1900 records complete... (35.13 s elapsed)
2000 records complete... (37.19 s elapsed)
2100 records complete... (38.89 s elapsed)
2200 records complete... (40.53 s elapsed)
2300 records complete... (42.7 

In [17]:
matrix_to_png(means_matrix_100).save("sparse_output_100.png")

In [22]:
means_matrix_1k=rough_means_matrix_sparse(ids_1k,movies,progress=True,prog_interval=500)

0 records complete... (0.0 s elapsed)
500 records complete... (8.986 s elapsed)
1000 records complete... (18.263 s elapsed)
1500 records complete... (31.373 s elapsed)
2000 records complete... (44.462 s elapsed)
2500 records complete... (52.763 s elapsed)
3000 records complete... (61.015 s elapsed)
3500 records complete... (70.127 s elapsed)
4000 records complete... (79.544 s elapsed)
4500 records complete... (91.191 s elapsed)
5000 records complete... (102.875 s elapsed)
5500 records complete... (116.065 s elapsed)
6000 records complete... (128.472 s elapsed)
6500 records complete... (139.548 s elapsed)
7000 records complete... (150.688 s elapsed)
7500 records complete... (159.704 s elapsed)
8000 records complete... (168.442 s elapsed)
8500 records complete... (177.344 s elapsed)
9000 records complete... (186.364 s elapsed)
9500 records complete... (194.789 s elapsed)
10000 records complete... (203.24 s elapsed)
10500 records complete... (211.694 s elapsed)
11000 records complete... (

89000 records complete... (1856.029 s elapsed)
89500 records complete... (1874.841 s elapsed)
90000 records complete... (1884.598 s elapsed)
90500 records complete... (1894.648 s elapsed)
91000 records complete... (1911.418 s elapsed)
91500 records complete... (1926.638 s elapsed)
92000 records complete... (1938.615 s elapsed)
92500 records complete... (1949.666 s elapsed)
93000 records complete... (1960.408 s elapsed)
93500 records complete... (1969.892 s elapsed)
94000 records complete... (1978.424 s elapsed)
94500 records complete... (1988.215 s elapsed)
95000 records complete... (1999.384 s elapsed)
95500 records complete... (2007.166 s elapsed)
96000 records complete... (2015.011 s elapsed)
96500 records complete... (2023.201 s elapsed)
97000 records complete... (2032.985 s elapsed)
97500 records complete... (2045.271 s elapsed)
98000 records complete... (2057.213 s elapsed)
98500 records complete... (2068.971 s elapsed)
99000 records complete... (2081.847 s elapsed)
99500 records

175000 records complete... (3722.616 s elapsed)
175500 records complete... (3733.294 s elapsed)
176000 records complete... (3743.52 s elapsed)
176500 records complete... (3752.898 s elapsed)
177000 records complete... (3761.538 s elapsed)
177500 records complete... (3770.388 s elapsed)
178000 records complete... (3779.169 s elapsed)
178500 records complete... (3789.682 s elapsed)
179000 records complete... (3802.69 s elapsed)
179500 records complete... (3818.321 s elapsed)
180000 records complete... (3833.3 s elapsed)
180500 records complete... (3843.574 s elapsed)
181000 records complete... (3853.94 s elapsed)
181500 records complete... (3864.68 s elapsed)
182000 records complete... (3873.703 s elapsed)
182500 records complete... (3882.909 s elapsed)
183000 records complete... (3893.912 s elapsed)
183500 records complete... (3905.643 s elapsed)
184000 records complete... (3918.005 s elapsed)
184500 records complete... (3931.41 s elapsed)
185000 records complete... (3942.468 s elapsed)

261000 records complete... (5603.654 s elapsed)
261500 records complete... (5617.981 s elapsed)
262000 records complete... (5629.761 s elapsed)
262500 records complete... (5642.051 s elapsed)
263000 records complete... (5653.183 s elapsed)
263500 records complete... (5662.153 s elapsed)
264000 records complete... (5672.905 s elapsed)
264500 records complete... (5685.429 s elapsed)
265000 records complete... (5694.853 s elapsed)
265500 records complete... (5704.825 s elapsed)
266000 records complete... (5717.621 s elapsed)
266500 records complete... (5729.663 s elapsed)
267000 records complete... (5739.679 s elapsed)
267500 records complete... (5749.291 s elapsed)
268000 records complete... (5760.716 s elapsed)
268500 records complete... (5771.06 s elapsed)
269000 records complete... (5780.34 s elapsed)
269500 records complete... (5790.202 s elapsed)
270000 records complete... (5805.536 s elapsed)
270500 records complete... (5817.186 s elapsed)
271000 records complete... (5826.896 s ela

346500 records complete... (7339.986 s elapsed)
347000 records complete... (7347.456 s elapsed)
347500 records complete... (7354.856 s elapsed)
348000 records complete... (7363.316 s elapsed)
348500 records complete... (7371.016 s elapsed)
349000 records complete... (7380.596 s elapsed)
349500 records complete... (7388.826 s elapsed)
350000 records complete... (7397.136 s elapsed)
350500 records complete... (7405.208 s elapsed)
351000 records complete... (7412.988 s elapsed)
351500 records complete... (7422.228 s elapsed)
352000 records complete... (7431.408 s elapsed)
352500 records complete... (7439.268 s elapsed)
353000 records complete... (7447.491 s elapsed)
353500 records complete... (7457.619 s elapsed)
354000 records complete... (7468.461 s elapsed)
354500 records complete... (7479.153 s elapsed)
355000 records complete... (7493.66 s elapsed)
355500 records complete... (7508.605 s elapsed)
356000 records complete... (7519.555 s elapsed)
356500 records complete... (7531.699 s el

432000 records complete... (9016.671 s elapsed)
432500 records complete... (9025.161 s elapsed)
433000 records complete... (9033.781 s elapsed)
433500 records complete... (9042.201 s elapsed)
434000 records complete... (9053.031 s elapsed)
434500 records complete... (9062.331 s elapsed)
435000 records complete... (9070.481 s elapsed)
435500 records complete... (9079.841 s elapsed)
436000 records complete... (9090.491 s elapsed)
436500 records complete... (9099.601 s elapsed)
437000 records complete... (9108.621 s elapsed)
437500 records complete... (9121.861 s elapsed)
438000 records complete... (9131.981 s elapsed)
438500 records complete... (9140.675 s elapsed)
439000 records complete... (9150.183 s elapsed)
439500 records complete... (9162.263 s elapsed)
440000 records complete... (9171.253 s elapsed)
440500 records complete... (9179.993 s elapsed)
441000 records complete... (9191.693 s elapsed)
441500 records complete... (9201.745 s elapsed)
442000 records complete... (9214.895 s e

In [26]:
matrix_to_png(means_matrix_1k).save("sparse_output_top_1k.png")

In [37]:
def get_predictions_sparse(input_list, matrix, id_list, title_ref=None):
    numrows = len(matrix)
    numcols = numrows
    predictions = []
    total=len(input_list)
    
    for m_r in input_list:
        movie_id = m_r[0]
        rating = m_r[1]
        if movie_id not in id_list:
            total-=1
            continue
        predict_row = matrix[id_list.index(movie_id)]
        predict_list = [np.interp(rating,[1,3,5],predict_row[i])*4+1 for i in range(numcols)]
        predictions.append(predict_list)
        
    predict_sums=[]
    for i in range(len(predictions[0])):
        predict_sums.append(0)
        for j in range(len(predictions)):
            predict_sums[i] += predictions[j][i]
    
    predictions_df = pd.DataFrame({"movie_id":id_list,"rating":[x/total for x in predict_sums]})
    predictions_df.query(f'movie_id not in {[x[0] for x in input_list]}', inplace=True)
    if title_ref is not None:
        predictions_df["title"]=[get_title(title_ref, i) for i in predictions_df["movie_id"]]
    
    return predictions_df.sort_values("rating",ascending=False)

In [44]:
get_predictions_sparse([[143,4],[175,5],[329,3],[17381,5],[14961,2]],means_matrix_1k, ids_1k, titles_df)

,movie_id,rating,title
795,14550,4.529149,The Shawshank Redemption: Special Edition
656,12293,4.489366,The Godfather
306,5582,4.451519,Star Wars: Episode V: The Empire Strikes Back
541,10042,4.436737,Raiders of the Lost Ark
778,14302,4.433321,The Sopranos: Season 2
896,16147,4.430721,The Sopranos: Season 1
847,15296,4.418826,Band of Brothers
314,5760,4.417933,The Sopranos: Season 3
453,8116,4.413639,The Sopranos: Season 4
905,16265,4.397985,Star Wars: Episode IV: A New Hope


In [63]:
get_predictions_sparse([[13673,5],[3962,5],[847,4],[7904,5],[14961,1],[5760,1]],means_matrix_1k, ids_1k, titles_df)

,movie_id,rating,title
795,14550,4.503558,The Shawshank Redemption: Special Edition
541,10042,4.434600,Raiders of the Lost Ark
306,5582,4.423479,Star Wars: Episode V: The Empire Strikes Back
656,12293,4.413385,The Godfather
905,16265,4.379472,Star Wars: Episode IV: A New Hope
693,12870,4.342953,Schindler's List
525,9628,4.324827,Star Wars: Episode VI: Return of the Jedi
395,6974,4.302436,The Usual Suspects
847,15296,4.297206,Band of Brothers
582,10947,4.289995,The Incredibles
